# Python for Analytics
__Name__: Jide Afolabi<br>
__Date__: March 3, 2024 <br />
__Project__: Python <br />
__Purpose__: Use of pandas to read and analyze a data file, calculating a correlation between test questions and overall performance.


### Objective
Using a combination of Pandas, NumPy, and native Python analyze a data file `P4-ScoreData.csv` which contains results for a quiz for which you will need to calculate the  _Discrimination_ statistic for each question, calculating a correlation between test questions and overall performance. Correlation may be applied to exam and quiz question results to determine how effective each question is at differentiating between students that did well and students that did not do well on the assessment. 

> _Discrimination_: Indicates how well a question differentiates between students who know the subject matter and those who don't. A question is a good discriminator when students who answer the question correctly also do well on the test. Values can range from -1.0 to +1.0. A question is flagged for review if its discrimination value is less than 0.1 or negative. Discrimination values can't be calculated when the question's difficulty score is 100% or when all students receive the same score on a question.

> Discrimination values are calculated with the Pearson correlation coefficient. `X` represents the scores of each student on a question and `Y` represents the scores of each student on the test.
 
> These variables are the standard score, sample mean, and sample standard deviation, respectively:

In [1]:
import pandas as pd
import numpy as np

# Read in the csv file
filename = 'P4-ScoreData.csv'
df = pd.read_csv(filename)
# Use .fillna to fill up the missing values with zeros
df.fillna(0, inplace=True)
# Use the .set_index to make the name column the index
df.set_index('Name', inplace=True)        
# Unpack the shape attribute and print the number of students and the number of questions
num_students, num_questions = df.shape
print('Number of students:', num_students)
print('Number of questions:', num_questions)
print()

# Calculate the mean score and standard deviation for each question
question_mean = df.mean(axis=0)
question_stddev = df.std(axis=0)

# Create a new column named Total and calculate the total score for each student 
df['Total'] = df.sum(axis=1)

# Calculate the overall quiz mean and standard deviation
Overall_quiz_mean = df['Total'].mean()
Overall_quiz_stddev = df['Total'].std()

# Create a new dataframe by slicing the previous DataFrame to include only the first 10 question columns
question_columns_df = df.iloc[:, :10]

# Initialize a DataFrame to store the discrimination statistics for each question
discrimination_stat_df = pd.DataFrame()
# Calculate the discrimination statistics for each question
for col in question_columns_df.columns:
    x_diff = (question_columns_df[col] - question_mean[col]) # Calculate the difference between student's score on a question vs question mean 
    x_term = x_diff / question_stddev[col] # Divide by the question's standard deviation
    
    y_diff = (df['Total'] - Overall_quiz_mean) # Calculate the difference between student's total on a question vs overall quiz mean
    y_term = y_diff / Overall_quiz_stddev # Divide by the overall standard deviation of the quiz
    
    discrimination_stat_df[col] = x_term * y_term # Multiply the x and y terms
print("Discrimination Statistics for each question:") # Print the discrimination statistics dataframe
print(discrimination_stat_df)

# Sum the product terms and divide by the number of students less 1 to get the discrimination statistics for each question
question_discrimination_stat = discrimination_stat_df.sum(axis=0) / (num_students - 1)

# Print the resulting question discrimination statistics
print('\nResulting question Discrimination statistics:')
print(question_discrimination_stat)

# Calculate the discrimination statistics using pandas corrwith function
print('\nDiscrimination statistics using alternative pandas correlation function corrwith')
alt_ques_discrimination_stat = question_columns_df.corrwith(df['Total'], axis = 0) 
print(alt_ques_discrimination_stat)

""" a. Which question appears to need review based on a negative Discrimination statistic? 
        Question 5 with a correlative coefficient of -0.258207 appears to needs to be reviewed and this is particularly because 
        students who had a better overall quiz performance did poorly in this question. 
        
    b. Why might this not be an appropriate interpretation given the actual student scores?
        A negative correlation, nevertheless, does not imply that the question in question 5 is flawed or that students who 
        performed better on the quiz performed worse on that specific question; rather, it may be attributable to other 
        unrelated variables.
"""

Number of students: 15
Number of questions: 10

Discrimination Statistics for each question:
                Q1        Q2        Q3        Q4        Q5        Q6  \
Name                                                                   
Sophia    0.145635 -0.054982  0.195744 -0.146537  0.079132  0.092482   
Muhammad  0.549442  1.763163  0.147698 -0.201034 -0.298544  0.959496   
Olivia    0.622260 -0.117461  0.167272  0.113838  0.169055  0.938478   
Aiden     3.137780  3.356382  3.655079  3.922580 -0.568313  4.317207   
Mia       0.218453  0.082473  0.234893  0.219805 -0.118698  0.121382   
Liam      0.635500 -0.679774  0.968044  0.077507  0.115101  0.638963   
Isabella  0.688458 -0.129957 -0.092534  1.070565  0.187040  0.218593   
Elijah    0.039719 -0.048734  0.021354  0.019982 -0.010791  0.034681   
Ava       0.443526  1.088387 -0.119226  1.379382 -3.373907  0.281648   
Mateo    -0.145635  0.357381 -0.156595  0.053286  0.079132 -0.080921   
Zoe       0.059578  0.007498 -0.026692 -0.0

' a. Which question appears to need review based on a negative Discrimination statistic? \n        Question 5 with a correlative coefficient of -0.258207 appears to needs to be reviewed and this is particularly because \n        students who had a better overall quiz performance did poorly in this question. \n        \n    b. Why might this not be an appropriate interpretation given the actual student scores?\n        A negative correlation, nevertheless, does not imply that the question in question 5 is flawed or that students who \n        performed better on the quiz performed worse on that specific question; rather, it may be attributable to other \n        unrelated variables.\n'

### Methodology
1. Used pandas `.read_csv` function to read the `P4-ScoreData.csv` file into a `DataFrame` and saved the data file in the same folder as my Notebook. This way, I can simply use the filename without additional path information when reading the file. 
2. A few students left some questions unanswered when taking the quiz. These appear as `NaN` values in the `DataFrame`. I used the `.fillna` method of the `DataFrame` to fill those missing values with zeros 
3. The student names are simply labels for the row-level data. Used the `.set_index` method of the `DataFrame` to turn the `Name` column into the index. Included the `inplace=True` argument to modify the `DataFrame` object instead of returning a copy. I had 15 students and 10 questions.
5. Calculated the mean score for each question using the `mean` method of the `DataFrame` and stored in a variable that I can reference later when calculating the Discrimination statistic. Using the default axis of the `DataFrame` produced a mean for each of the 10 questions. 
6. Similarly, calculated the sample standard deviation for each question using the `std` method of the `DataFrame` and stored in a variable that I can reference later when calculating the Discrimination statistic. Using the default axis of the `DataFrame` produced a standard deviation for each of the 10 questions. 
7. Added a new column named `Total` to the `DataFrame` using the `sum` method of the `DataFrame`, specifying `axis=1`. Printed the `DataFrame` after this step to ensure that each student's total score for the quiz was calculated correctly.
8. Calculated the overall quiz mean by calling the `mean` method on the `DataFrame`'s `Total` column and stored in a variable that I can reference later when calculating the Discrimination statistic. 
9. Similarly, calculated the sample standard deviation of the overall quiz scores by calling the `std` method on the `DataFrame`'s `Total` column and stored it in a variable that I can reference later when calculating the Discrimination statistic. 
10. For the next step, I isolated the 10 question columns. An easy way to do this was to create an index by slicing the `columns` attributed to include only the first 10 columns.
11. To finish the calculation of the Discrimation statistic step-by-step for each question, I calculated the difference between each student's score on that question and the question's mean. This difference was then divided by that question's standard deviation. This is the term involving the `X`'s in the equation above.
12. In addition, I calculatedd the difference between each student's total score on the quiz and the overall quiz mean. This difference was then divided by the sample standard deviation of the overall quiz scores. This is the term involving the `Y`'s in the equation above.
13. Next, calculated the product of these two term variables I created in the previous two steps. I used the Transposition of score differences constructed in Step 11 above when multiplying.
14. Finally, to complete the calculation of the Discrimation statistic for each question, I sumed the Transposition of the product calculated in the previous step and divide that sum by `num_students-1`. Printed the resulting question Discrimination statistics.
15. A much simpler way to calculate the Discrimination statistic was simply to use pandas `corrwith` method to calculate the Pearson correlation coefficient for all the quiz questions correlated with the `Total` column calculated in Step 7 above. This should result in the same Discrimination statistic for each of the quiz questions but without needing Steps 8-14 above. Print this result and compare to the values you calculated step-by-step earlier. Before printing, I droped the `Total` from the Series produced as my result (as it will obviously correlate perfectly with itself). 

### Summary
Using the information my analysis yielded, I answered the following at the bottom of my code.<br />a. Which question appears to need review based on a negative Discrimination statistic (suggesting students who did better on the quiz overall actually did worse on this question)?<br />b. Why might this not be an appropriate interpretation given the actual student scores?